In [21]:
import pandas as pd
electoral_result = pd.read_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/1976-2020-president.csv')

electoral_result = electoral_result.drop(columns=['state_po', 'state_fips', 'state_cen', 'state_ic', 'writein', 'notes', 'version'])

electoral_result = electoral_result[electoral_result['year'] >= 2008]

electoral_result.to_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/2008-2020-president.csv')
print(electoral_result.head())


      year    state        office         candidate party_detailed  \
2728  2008  ALABAMA  US PRESIDENT      MCCAIN, JOHN     REPUBLICAN   
2729  2008  ALABAMA  US PRESIDENT  OBAMA, BARACK H.       DEMOCRAT   
2730  2008  ALABAMA  US PRESIDENT               NaN    INDEPENDENT   
2731  2008  ALABAMA  US PRESIDENT               NaN            NaN   
2732  2008   ALASKA  US PRESIDENT      MCCAIN, JOHN     REPUBLICAN   

      candidatevotes  totalvotes party_simplified  
2728         1266546     2099819       REPUBLICAN  
2729          813479     2099819         DEMOCRAT  
2730           16089     2099819            OTHER  
2731            3705     2099819            OTHER  
2732          193841      326197       REPUBLICAN  


In [22]:
winning_percentage = round((electoral_result['candidatevotes'] / electoral_result['totalvotes']) * 100, 2)
electoral_result['winning_percentage'] = winning_percentage


In [23]:
import pandas as pd

# Drop only the columns that exist in the DataFrame
# columns_to_drop = ['candidatevotes', 'totalvotes', 'party_simplified']
# existing_columns_to_drop = [col for col in columns_to_drop if col in electoral_result.columns]
# electoral_result = electoral_result.drop(columns=existing_columns_to_drop)

filtered_electoral_result = electoral_result[electoral_result['party_detailed'].isin(['REPUBLICAN', 'DEMOCRAT'])]

df = pd.DataFrame(filtered_electoral_result)

# Reshape the DataFrame
merged = df.groupby(['year', 'state', 'office']).apply(lambda group: pd.Series({
    'Winning Candidate': group.loc[group['winning_percentage'].idxmax(), 'candidate'],
    'Winning Party': group.loc[group['winning_percentage'].idxmax(), 'party_detailed'],
    'Winning Percentage': group['winning_percentage'].max(),
    'Losing Candidate': group.loc[group['winning_percentage'].idxmin(), 'candidate'],
    'Losing Party': group.loc[group['winning_percentage'].idxmin(), 'party_detailed'],
    'Losing Percentage': group['winning_percentage'].min()
})).reset_index()

# Rename several rows to fit the format of the other datasets
merged = merged.rename(columns={'state': 'State', 'office': 'Office', 'year': 'Year'})

# Calculate overall United States results for each election year
overall_us_results = df.groupby(['year', 'party_detailed']).agg({
    'candidatevotes': 'sum',
    'totalvotes': 'sum'
}).reset_index()

# Calculate the percentage of votes for each party
overall_us_results['percentage'] = (overall_us_results['candidatevotes'] / overall_us_results['totalvotes']) * 100

# Pivot the overall results to get the winning and losing parties
pivoted_us_results = overall_us_results.pivot(index='year', columns='party_detailed', values=['candidatevotes', 'percentage']).reset_index()
pivoted_us_results.columns = ['Year', 'Democrat Votes', 'Republican Votes', 'Democrat Percentage', 'Republican Percentage']

# Determine the winning and losing parties
pivoted_us_results['Winning Party'] = pivoted_us_results.apply(lambda row: 'DEMOCRAT' if row['Democrat Votes'] > row['Republican Votes'] else 'REPUBLICAN', axis=1)
pivoted_us_results['Winning Candidate'] = pivoted_us_results.apply(lambda row: df[(df['year'] == row['Year']) & (df['party_detailed'] == row['Winning Party'])]['candidate'].values[0], axis=1)
pivoted_us_results['Winning Percentage'] = pivoted_us_results.apply(lambda row: row['Democrat Percentage'] if row['Winning Party'] == 'DEMOCRAT' else row['Republican Percentage'], axis=1)
pivoted_us_results['Losing Party'] = pivoted_us_results.apply(lambda row: 'REPUBLICAN' if row['Winning Party'] == 'DEMOCRAT' else 'DEMOCRAT', axis=1)
pivoted_us_results['Losing Candidate'] = pivoted_us_results.apply(lambda row: df[(df['year'] == row['Year']) & (df['party_detailed'] == row['Losing Party'])]['candidate'].values[0], axis=1)
pivoted_us_results['Losing Percentage'] = pivoted_us_results.apply(lambda row: row['Republican Percentage'] if row['Winning Party'] == 'DEMOCRAT' else row['Democrat Percentage'], axis=1)

# Add the United States row to the merged DataFrame
us_results = pivoted_us_results[['Year', 'Winning Candidate', 'Winning Party', 'Winning Percentage', 'Losing Candidate', 'Losing Party', 'Losing Percentage']]
us_results['State'] = 'UNITED STATES'
us_results['Office'] = 'US PRESIDENT'

# Append the United States results to the merged DataFrame
merged = pd.concat([merged, us_results], ignore_index=True)

# Display the merged DataFrame
print(merged)

     Year          State        Office    Winning Candidate Winning Party  \
0    2008        ALABAMA  US PRESIDENT         MCCAIN, JOHN    REPUBLICAN   
1    2008         ALASKA  US PRESIDENT         MCCAIN, JOHN    REPUBLICAN   
2    2008        ARIZONA  US PRESIDENT         MCCAIN, JOHN    REPUBLICAN   
3    2008       ARKANSAS  US PRESIDENT         MCCAIN, JOHN    REPUBLICAN   
4    2008     CALIFORNIA  US PRESIDENT     OBAMA, BARACK H.      DEMOCRAT   
..    ...            ...           ...                  ...           ...   
203  2020        WYOMING  US PRESIDENT     TRUMP, DONALD J.    REPUBLICAN   
204  2008  UNITED STATES  US PRESIDENT     OBAMA, BARACK H.      DEMOCRAT   
205  2012  UNITED STATES  US PRESIDENT     OBAMA, BARACK H.      DEMOCRAT   
206  2016  UNITED STATES  US PRESIDENT     CLINTON, HILLARY      DEMOCRAT   
207  2020  UNITED STATES  US PRESIDENT  BIDEN, JOSEPH R. JR      DEMOCRAT   

     Winning Percentage     Losing Candidate Losing Party  Losing Percentag

/tmp/ipykernel_5624/1003398760.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged = df.groupby(['year', 'state', 'office']).apply(lambda group: pd.Series({
/tmp/ipykernel_5624/1003398760.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_results['State'] = 'UNITED STATES'
/tmp/ipykernel_5624/1003398760.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [ ]:
# Drop only the columns that exist in the DataFrame
columns_to_drop = ['candidatevotes', 'totalvotes', 'party_simplified']
existing_columns_to_drop = [col for col in columns_to_drop if col in electoral_result.columns]
electoral_result = electoral_result.drop(columns=existing_columns_to_drop)

filtered_electoral_result = electoral_result[electoral_result['party_detailed'].isin(['REPUBLICAN', 'DEMOCRAT'])]


df = pd.DataFrame(filtered_electoral_result)

# Reshape the DataFrame
merged = df.groupby(['year', 'state', 'office']).apply(lambda group: pd.Series({
    'Winning Candidate': group.loc[group['winning_percentage'].idxmax(), 'candidate'],
    'Winning Party': group.loc[group['winning_percentage'].idxmax(), 'party_detailed'],
    'Winning Percentage': group['winning_percentage'].max(),
    'Losing Candidate': group.loc[group['winning_percentage'].idxmin(), 'candidate'],
    'Losing Party': group.loc[group['winning_percentage'].idxmin(), 'party_detailed'],
    'Losing Percentage': group['winning_percentage'].min()
})).reset_index()


# Rename several rows to fit the format of the other datasets
merged = merged.rename(columns={'state': 'State', 'office': 'Office', 'year': 'Year'})
# Display the merged DataFrame
print(merged)
merged.to_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/processed_electoral_data.csv', index=False)